In [33]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import iob_ia.utils.workflows2d as wf

### 1. Specify a path to an image

In [3]:
# For windows users: use "/" instead of "\" for file paths!
path = "G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2/20250520_10x_Nq146um_405_488_640_01.vsi"

### 2. Start by creating an object for checking an image using 2D segmentation
I called the object "bi", and told it I want to segment 2 channels:
- segment_channels=[1,3] means the 2nd and the 4th channels, since we use 0-based channel indexing

In [4]:
# We create an object for 2D segmentation and we'll tell it, which channels need to be segmented
bi = wf.Iob2DImageCheck(path=path, segment_channels=[1,3])

G:\20241120_IOB_Magdalena\iob-ia\src\iob_ia\utils\workflows2d.py:81: UserWarning: Voxel size is missing. Image metadata is = PhysicalPixelSizes(Z=None, Y=0.6500000000000001, X=0.6500000000000001).Please provide the voxel-size via set_voxel_size((Z, Y, X)).
  warnings.warn(
G:\20241120_IOB_Magdalena\iob-ia\src\iob_ia\utils\workflows2d.py:150: UserWarning: You requested segmentations for channels [1, 3]. But your image has only 3 channels. Set the channels to segment using 0-indexing for channel numbers. With e.g. set_segment_channels([1, 2]) for the last 2 channels of a 3-channel image.
  warnings.warn(


I got two warnings, which need to be taken care of:
1. somehow the image metada is incorrect, actually it misses the pixel size in the Z dimension (it says 'None')
2. I requested to segment the 2nd and 4th channel, but there are only 3 channels in total
Let's correct it:

In [5]:
# Specify the voxel size (Z, Y, X)
bi.set_voxel_size((1.46, 0.65, 0.65))

G:\20241120_IOB_Magdalena\iob-ia\src\iob_ia\utils\workflows2d.py:150: UserWarning: You requested segmentations for channels [1, 3]. But your image has only 3 channels. Set the channels to segment using 0-indexing for channel numbers. With e.g. set_segment_channels([1, 2]) for the last 2 channels of a 3-channel image.
  warnings.warn(


In [6]:
# Now, indicate to segment the 2nd and 3rd channel
bi.set_segment_channels([1,2])

### 3. Let's analyse the image, this will take some time and we'll see a lot of information

In [11]:
bi.analyse()

Loading image to memory...
2025-09-08 10:22:00,021 [INFO] ** TORCH CUDA version installed and working. **
creating new log file
2025-09-08 10:22:00,023 [INFO] WRITING LOG OUTPUT TO C:\Users\Microscopy\.cellpose\run.log
2025-09-08 10:22:00,023 [INFO] 
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.11.13 
torch version:  	2.8.0+cu126
>>> GPU activated: True
2025-09-08 10:22:00,024 [WARNING] model_type argument is not used in v4.0.1+. Ignoring this argument...
2025-09-08 10:22:00,026 [INFO] ** TORCH CUDA version installed and working. **
2025-09-08 10:22:00,027 [INFO] >>>> using GPU (CUDA)
2025-09-08 10:22:02,295 [INFO] >>>> loading model C:\Users\Microscopy\.cellpose\models\cpsam
2025-09-08 10:22:13,464 [WARNING] Resizing is depricated in v4.0.1+
2025-09-08 10:22:23,874 [WARNING] Resizing is depricated in v4.0.1+
2025-09-08 10:22:34,443 [WARNING] Resizing is depricated in v4.0.1+
2025-09-08 10:22:45,095 [WARNING] Resizing is depricated in v4.0.1+
2025-09-08 10:22:55

----
### Results
#### About the analysis
Cellpose object detection is good, but not great. It needs some filtering:
I hardcoded the size filtering to 42-233um^2 (2D area, empiric values).
Additionally, I also wanted to exclude object with low fluorescence signal. Therfore, the mean intensity of all objects is averaged and objects with mean intensities below the average of "all object mean intensity" are filtered out (per channel).

Checking for double positivity is done only if exactly 2 channels were segmented. It will check if objects from one channel have at least 50% overlap with objects of the other channel.

#### Result tabel
Indicates the counts.

In [12]:
# A nicer visualisation of the table can be gotten like:
bi.result_table

[autoreload of fsspec.spec failed: Traceback (most recent call last):
  File "C:\Users\Microscopy\.conda\envs\iob-ia\Lib\site-packages\IPython\extensions\autoreload.py", line 325, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\Microscopy\.conda\envs\iob-ia\Lib\site-packages\IPython\extensions\autoreload.py", line 621, in superreload
    update_generic(old_obj, new_obj)
  File "C:\Users\Microscopy\.conda\envs\iob-ia\Lib\site-packages\IPython\extensions\autoreload.py", line 447, in update_generic
    update(a, b)
  File "C:\Users\Microscopy\.conda\envs\iob-ia\Lib\site-packages\IPython\extensions\autoreload.py", line 415, in update_class
    update_instances(old, new)
  File "C:\Users\Microscopy\.conda\envs\iob-ia\Lib\site-packages\IPython\extensions\autoreload.py", line 373, in update_instances
    object.__setattr__(ref, "__class__", new)
TypeError: can't apply this __setattr__ to _Cached object
]


,Channel,Cellpose objects,Filtered objects,Double-positive objects
0,Channel 1: 488 C,6282,5813,2448
1,Channel 2: 640 C,7333,6145,2700


### Save the results
Will save the masks, image channels (that were segmented) and the result table.

In [9]:
bi.save_results()

Saved labels to: G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2\20250520_10x_Nq146um_405_488_640_01_IOB-2D-workflow_cp-mask_ch1-488 C.tif
Saved mask image to G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2\20250520_10x_Nq146um_405_488_640_01_IOB-2D-workflow_cp-mask_ch1-488 C.tif.
Saved labels to: G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2\20250520_10x_Nq146um_405_488_640_01_IOB-2D-workflow_cp-mask_ch2-640 C.tif
Saved mask image to G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2\20250520_10x_Nq146um_405_488_640_01_IOB-2D-workflow_cp-mask_ch2-640 C.tif.
Saved labels to: G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2\20250520_10x_Nq146um_405_488_640_01_IOB-2D-workflow_filtered-mask_ch1-488 C.tif
Saved mask image to G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2\20250520_10x_Nq146um_405_488_640_01_IOB-2D-workflow_filtered-mask_ch1-488 C.tif.
Saved labels to: G:/20241120_IOB_Magdalena/20250520_TestClearing/Organoid 2\

----
### Retrieve results directly
You can get the results from the "bi" object directly (e.g. for visualisation in napari), using following functions:
- get_channel() -> The segmented channel image (stack of maxIP substacks)
- get_cp_mask() -> The raw cellpose segmentation
- get_filtered_mask() -> The filtered cell objects
- get_double_pos_mask() -> The double positive objects

each of these functions has a channel parameter, i.e. the channel you want to get. Basically, provide the same 0-based channel index you used at the beginning, for defining which channels to segment.

It will return 2 values:
1. image data
2. voxel size of the image file

In [13]:
image, sizeOfVoxel = bi.get_channel(channel=2)

In [14]:
cpMask, sizeOfVoxel = bi.get_cp_mask(channel=1)

In [15]:
fMask, sizeOfVoxel = bi.get_filtered_mask(channel=1)

In [16]:
dpMask, sizeOfVoxel = bi.get_double_pos_mask(channel=1)

----
## Viszalise in napari
After retrieving the image data, we can visualise it in napari. E.e.:

In [46]:
import iob_ia.utils.visualise as vis

In [48]:
# Adding a channel image
img_gfp, sizeOfVoxel = bi.get_channel(channel=2)
vis.add_image(img_gfp, name="GFP_channel", colormap="green", scale=sizeOfVoxel)

In [49]:
# Add a segmentation mask
filtered_gfp, sizeOfVoxel = bi.get_filtered_mask(channel=2)
vis.add_labels(filtered_gfp, name="Filtered_GFP_objects", scale=sizeOfVoxel)

2025-09-08 11:25:12,180 [INFO] No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'
